In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow
import sys
import os
sys.path.append(os.path.abspath('..'))

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer
from lightgbm import LGBMClassifier
from sklearn.pipeline import make_pipeline

from src.utils import *
from src.aeroclub import *
from src.models import *

In [ ]:
train = pd.read_parquet('../data/train.parquet')
print("Train shape:", train.shape)
test = pd.read_parquet('../data/test.parquet')
print("Test shape:", test.shape)

In [ ]:
train_df = safe_optimize_dtypes(train)

In [ ]:
print('=== Basic EDA ===')
print(f"Total Users: {train['profileId'].nunique():,}")
print(f"Total Items: {train['Id'].nunique():,}")
print(f"Total Sessions: {train['ranker_id'].nunique():,}")
# 한번의 검색에서 사용자가 제시받는 항공편 옵션들 = 1 session
print(f"Select Rate: {train['selected'].mean():.4f}")

# 세션당 후보 수 분포
session_sizes = train.groupby('ranker_id').size()
# ranker_id(1개의 세션)의 크기 -> 몇 개의 항공편이 제시되었는지 확인
print(f"\nPer Session: {session_sizes.describe()}")

In [ ]:
sns.countplot(data=train, x='selected')
plt.title('Target Data Distribution')
plt.show()

In [ ]:
cat_cols, num_cols, date_cols, id_cols = get_column_types(train)

In [ ]:
mean_score, std_score, scores = run_baseline_cv(train, cat_cols, date_cols=date_cols)

In [1]:
# s-kfold 사용 x